# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [10]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [11]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,margaret river,-33.9500,115.0667,56.89,87,0,10.71,AU,1710185889
1,1,grytviken,-54.2811,-36.5092,45.16,74,61,12.66,GS,1710185889
2,2,sitka,57.0531,-135.3300,39.09,65,75,8.05,US,1710185889
3,3,tsevie,6.4261,1.2133,83.28,66,89,9.42,TG,1710185889
4,4,port saint john's,-31.6229,29.5448,77.77,85,0,16.04,ZA,1710185889


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [12]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng", 
    "Lat",
    geo = True,
    tiles = "OSM"
  )

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df2 = city_data_df[(city_data_df["Max Temp"] > 20) & 
                            (city_data_df["Max Temp"] < 30) & 
                            (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
city_data_df_drop = city_data_df2.dropna()

# Display sample data
city_data_df_drop

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
147,147,meadow lakes,61.6247,-149.6011,25.32,89,0,0.00,US,1709993949
164,164,kodiak,57.7900,-152.4072,27.91,74,0,13.80,US,1709993952
233,233,fallon,39.4735,-118.7774,22.87,86,0,6.91,US,1709993964
255,255,swift current,50.2834,-107.8014,23.81,80,0,12.66,CA,1709993968
372,372,fort mcmurray,56.7268,-111.3810,26.19,63,0,9.22,CA,1709993987
439,439,hoemul-li,41.4339,129.6700,29.37,45,0,3.49,KP,1709993998
521,521,brookings,44.3114,-96.7984,24.19,79,0,6.91,US,1709993924
530,530,la junta,37.9850,-103.5438,25.09,86,0,8.05,US,1709994014
542,542,pierre,44.3683,-100.3510,20.32,86,0,0.00,US,1709994016


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df_drop[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]= ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
147,meadow lakes,US,61.6247,-149.6011,89,
164,kodiak,US,57.7900,-152.4072,74,
233,fallon,US,39.4735,-118.7774,86,
255,swift current,CA,50.2834,-107.8014,80,
372,fort mcmurray,CA,56.7268,-111.3810,63,
439,hoemul-li,KP,41.4339,129.6700,45,
521,brookings,US,44.3114,-96.7984,79,
530,la junta,US,37.9850,-103.5438,86,
542,pierre,US,44.3683,-100.3510,86,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories" :"accommodation.hotel",
"apiKey": geoapify_key,
         "limit" : 20}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
   
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?PARAMS"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
meadow lakes - nearest hotel: Best Western Lake Lucille Inn
kodiak - nearest hotel: No hotel found
fallon - nearest hotel: Value Inn
swift current - nearest hotel: Days Inn
fort mcmurray - nearest hotel: Nomad Hotel and Suites
hoemul-li - nearest hotel: No hotel found
brookings - nearest hotel: Quality Inn
la junta - nearest hotel: Travel Inn of La Junta
pierre - nearest hotel: Quality Inn


,City,Country,Lat,Lng,Humidity,Hotel Name
147,meadow lakes,US,61.6247,-149.6011,89,Best Western Lake Lucille Inn
164,kodiak,US,57.7900,-152.4072,74,No hotel found
233,fallon,US,39.4735,-118.7774,86,Value Inn
255,swift current,CA,50.2834,-107.8014,80,Days Inn
372,fort mcmurray,CA,56.7268,-111.3810,63,Nomad Hotel and Suites
439,hoemul-li,KP,41.4339,129.6700,45,No hotel found
521,brookings,US,44.3114,-96.7984,79,Quality Inn
530,la junta,US,37.9850,-103.5438,86,Travel Inn of La Junta
542,pierre,US,44.3683,-100.3510,86,Quality Inn


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [14]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    hover_cols =["Hotel Name","Country"],
    geo = True,
    tiles = "OSM"
    
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Country)